In [2]:
import pandas as pd
import numpy as np
import re
import math
import geopandas as gpd
from shapely.geometry import Point, Polygon
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
import math

In [3]:
def df_clean(df):
    #reset_index
    df.reset_index(inplace=True)


    #turn price into int
    #df["price"] = df["price"].str.replace(",","")
    #df["price"] = df["price"].str.replace("$","")
    #df["price"] = df["price"].astype("int")

    
    #drop duplicate rows on URL
    df = df.drop_duplicates("url")
    
    #reset_index\n",
    df.reset_index(inplace=True)
   
    #turn price into int
    df["price"] = df["price"][df["price"].notnull()].str.replace(",","")
    df["price"] = df["price"][df["price"].notnull()].str.replace("$","")
    df["price"] = df["price"][df["price"].notnull()].astype("int")
    
    #shipping price
    df["shipping_price"] = df["shipping_price"][df["shipping_price"].notnull()].apply(lambda x: x.split(" ")[1])
    df["shipping_price"] = df["shipping_price"][df["shipping_price"].notnull()].apply(lambda x: x.replace("Shipping", "0")
                                                                                      .replace("$","")
                                                                                      .replace(".",""))
                                                                                     
    df["shipping_price"] = df["shipping_price"][df["shipping_price"].notnull()].replace('Local', np.nan)
    df["shipping_price"] = df["shipping_price"][df["shipping_price"].notnull()].apply(lambda x: float(x)/100)



    #category

    df["category"] = df["category"][df["category"].notnull()].apply(lambda x: x
                                                                    .replace("[","")
                                                                    .replace("]","")
                                                                    .replace("'","")
                                                                    .replace("\\",""))
    df["category"] = df["category"][df["category"].notnull()].apply(lambda x: x.split(", "))
   
    #buld category tiers
    df["cat_length_of_1"] = df["category"][df["category"].notnull()].apply(lambda x: len(x)>=2)
    df["cat_length_of_2"] = df["category"][df["category"].notnull()].apply(lambda x: len(x)>=3)
    df["cat_length_of_3"] = df["category"][df["category"].notnull()].apply(lambda x: len(x)>=4)
    df["cat_length_of_4"] = df["category"][df["category"].notnull()].apply(lambda x: len(x)>=5)
    df["cat_length_of_5"] = df["category"][df["category"].notnull()].apply(lambda x: len(x)>=6)

    df["category_1"] = df["category"][df["category"].notnull()].apply(lambda x: x[0] if x[0]!="" else np.nan)

    #df["shipping_price"] = df["shipping_price"].apply(lambda x: int(x.replace("$","")
    #                                                                .replace(".","")
    #                                                                .split(" ")[1])/100 if x!="Free Shipping" else 0)


    #category
#     df["category"] = df["category"][df["category"].notnull()].apply(lambda x: x
#                                                                     .replace("[","")
#                                                                     .replace("]","")
#                                                                     .replace("'","")
#                                                                     .replace("\\",""))
                                                                             
#     df["category"] = df["category"][df["category"].notnull()].apply(lambda x: x.split(", "))
                                                                                                                                        
    #build category tiers
    df["cat_length_of_1"] = df["category"][df["category"].notnull()].apply(lambda x: len(x)>=2)
    df["cat_length_of_2"] = df["category"][df["category"].notnull()].apply(lambda x: len(x)>=3)
    df["cat_length_of_3"] = df["category"][df["category"].notnull()].apply(lambda x: len(x)>=4)
    df["cat_length_of_4"] = df["category"][df["category"].notnull()].apply(lambda x: len(x)>=5)
    df["cat_length_of_5"] = df["category"][df["category"].notnull()].apply(lambda x: len(x)>=6)


    df["category_1"] = df["category"][df["category"].notnull()].apply(lambda x: x[0] if x[0]!="" else np.nan)

    #df["category_1"] = df["category"].apply(lambda x: x[0] if x[0]!="" else np.nan)

    df["category_2"] = df[df["cat_length_of_1"]==True]["category"].apply(lambda x: x[1])
    df["category_3"] = df[df["cat_length_of_2"]==True]["category"].apply(lambda x: x[2])
    df["category_4"] = df[df["cat_length_of_3"]==True]["category"].apply(lambda x: x[3])
    df["category_5"] = df[df["cat_length_of_4"]==True]["category"].apply(lambda x: x[4])
    df["category_6"] = df[df["cat_length_of_5"]==True]["category"].apply(lambda x: x[5])

    df.drop(columns=["cat_length_of_1","cat_length_of_2","cat_length_of_3","cat_length_of_4","cat_length_of_5"],axis=1, inplace=True)
    
    #clean up original price
    df["original_price"] = df["original_price"].apply(lambda x: np.nan if type(x)==np.float else int(x.replace("$","").replace(",","")))

    #clean up sold status and turn into binary


    #clean up sold status and turn into binary
    df["sold_status"] = df["sold_status"].apply(lambda x: "Sold" if x=="Item sold" else "Not Sold")
    df["sold_status"] = df["sold_status"].apply(lambda x: 0 if x=="Not Sold" else 1)
                                                                                                                                        
    #location
    df["location"] = df["location"][df["location"].notnull()].apply(lambda x: x
                                                                    .replace("['Local | $10.99']","")
                                                                    .replace("[]","")
                                                                    .split("|")[-1]
                                                                    .replace("from","")
                                                                    .replace("']","")
                                                                    .strip())
    
    df["location"] = df["location"].replace("", np.nan)

    #clean up sold status

    df["sold_status"] = df["sold_status"].apply(lambda x: "Sold" if x=="Item sold" else "Not Sold")
    df["sold_status"] = df["sold_status"].apply(lambda x: 0 if x=="Not Sold" else 1)
    

    #User_Id
    df["user_id"] = df["user_id"][df["user_id"].notnull()].apply(lambda x: x.replace("Sold by ",""))
    
    #profile verified
    df["profile_verified"] = df["profile_verified"][df["profile_verified"].notnull()].apply(lambda x: x.split(",")[-1]
                                                                                            .replace("'","")
                                                                                            .replace("]","")
                                                                                            .replace("sales","Not Verified")
                                                                                            .replace("sale","Not Verified").strip())
    
    #badges
    df["badges"] = df["badges"][df["badges"].notnull()].apply(lambda x: list(set(re.findall("badge_(.+?).svg", x))))
    
    df["reliable"] = df["badges"][df["badges"].notnull()].apply(lambda x: 1 if "reliable" in x else 0)
    df["speedy_shipper"] = df["badges"][df["badges"].notnull()].apply(lambda x: 1 if "speedy_shipping" in x else 0)
    df["fast_response"] = df["badges"][df["badges"].notnull()].apply(lambda x: 1 if "fast_response" in x else 0)
    df["member_since"] = df["badges"][df["badges"].notnull()].apply(lambda x: int("".join(re.findall("\d","".join(x)))))
    
    #responsiveness
    df["response_within_24hours"] = df["responsive"][df["responsive"].notnull()].apply(lambda x: 1 if x=="This seller usually responds within 24 hours" else 0)
    
                                                                                                                                        
    #discount

    df["discount"] = abs((df["price"] - df["original_price"])/df["original_price"])*100
    df["discount"] = df["discount"].replace(np.nan, 0)
    
    #convert posted date to datetime object
    df["posted_date"] = pd.to_datetime(df["posted_date"])
    
    #profile verified - convert to binary
    df["profile_verified"] = df["profile_verified"][df["profile_verified"].notnull()].apply(lambda x: 1 if x=="Profile verified" else 0 )

    df["profile_verified"] = df["profile_verified"].apply(lambda x: 1 if x=="Profile verified" else 0 )

    
    
    #drop duplicate rows on URL
    #df = df.drop_duplicates("url")


    return df
    

In [4]:
df = pd.read_csv("/Users/tonypark/Desktop/data/Mercarie_data2.csv")
pd.set_option('display.max_columns', None)


In [5]:
df1 = pd.read_csv("/Users/tonypark/Desktop/data/Mercari_data2.csv")
df2 = pd.read_csv("/Users/tonypark/Desktop/data/Mercarie_data1.csv")
df3 = pd.read_csv("/Users/tonypark/Desktop/data/Mercarie_data10.csv")
df4 = pd.read_csv("/Users/tonypark/Desktop/data/Mercarie_data2.csv")
df5 = pd.read_csv("/Users/tonypark/Desktop/data/Mercarie_data3.csv")
df6 = pd.read_csv("/Users/tonypark/Desktop/data/Mercarie_data5.csv")
df7 = pd.read_csv("/Users/tonypark/Desktop/data/Mercarie_data6.csv")
df8 = pd.read_csv("/Users/tonypark/Desktop/data/Mercarie_data7.csv")
df9 = pd.read_csv("/Users/tonypark/Desktop/data/Mercarie_data8.csv")
df10 = pd.read_csv("/Users/tonypark/Desktop/data/Mercarie_data9.csv")
df11 = pd.read_csv("/Users/tonypark/Desktop/data/data1.csv")
df12 = pd.read_csv("/Users/tonypark/Desktop/data/data123.csv")
df13 = pd.read_csv("/Users/tonypark/Desktop/data/data13.csv")
df14 = pd.read_csv("/Users/tonypark/Desktop/data/data2.csv")
df15 = pd.read_csv("/Users/tonypark/Desktop/data/data3.csv")
df16 = pd.read_csv("/Users/tonypark/Desktop/data/data321.csv")
df17 = pd.read_csv("/Users/tonypark/Desktop/data/data4.csv")
df18 = pd.read_csv("/Users/tonypark/Desktop/data/data5.csv")
df19 = pd.read_csv("/Users/tonypark/Desktop/data/data6.csv")
df20 = pd.read_csv("/Users/tonypark/Desktop/data/data_2Q.csv")
df21 = pd.read_csv("/Users/tonypark/Desktop/data/data_4Q.csv")
df22 = pd.read_csv("/Users/tonypark/Desktop/data/data_firstQ.csv")
df23 = pd.read_csv("/Users/tonypark/Desktop/data/data_thirdQ.csv")
df24 = pd.read_csv("/Users/tonypark/Desktop/data/names.csv")
df25 = pd.read_csv("/Users/tonypark/Downloads/OfferUpProject/data_2Q")
df26 = pd.read_csv("/Users/tonypark/Downloads/OfferUpProject/data_firstQ.csv")
df27 = pd.read_csv("/Users/tonypark/Downloads/OfferUpProject/data_thirdQ.csv")
df28 = pd.read_csv("/Users/tonypark/Downloads/OfferUpProject/data_4Q.csv")
df29 = pd.read_csv("/Users/tonypark/Downloads/OfferUpProject/data123.csv")
df30 = pd.read_csv("/Users/tonypark/Downloads/OfferUpProject/1Q_data")
df31 = pd.read_csv("/Users/tonypark/Downloads/OfferUpProject/data123.csv")
df32 = pd.read_csv("/Users/tonypark/Downloads/OfferUpProject/data321.csv")
df33 = pd.read_csv("/Users/tonypark/Downloads/OfferUpProject/data13.csv")
df34 = pd.read_csv("/Users/tonypark/Downloads/OfferUpProject/data2.csv")
df35 = pd.read_csv("/Users/tonypark/Downloads/OfferUpProject/data6.csv")
df36 = pd.read_csv("/Users/tonypark/Downloads/OfferUpProject/data1.csv")
df37 = pd.read_csv("/Users/tonypark/Downloads/OfferUpProject/data4.csv")
df38 = pd.read_csv("/Users/tonypark/Downloads/OfferUpProject/data3.csv")
df39 = pd.read_csv("/Users/tonypark/Downloads/OfferUpProject/Mercarie_data3.csv")
df40 = pd.read_csv("/Users/tonypark/Downloads/OfferUpProject/Mercarie_data1.csv")

/Users/tonypark/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (12,13) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/tonypark/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (12) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/tonypark/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
pdList = [df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12, df13, df14, df15, df16, df17, df18
         , df19, df20, df21, df22, df23, df24, df25, df26, df27, df28, df29, df30, df31, df32, df33, df34
         ,df35,df36, df37, df38, df39, df40]

In [7]:
new_df = pd.concat(pdList)

In [8]:
cleaned_new_df = df_clean(new_df)

/Users/tonypark/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)
/Users/tonypark/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
/Users/tonypark/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4908: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors

In [10]:
cleaned_new_df.to_csv("total_data.csv")